In [54]:
import numpy as np
import itertools

In [14]:
num_pulleys = 4

### Representation used for this code development ###

![](string_coordinate_representation_img_1.JPG)

#### `get_li` ####

For getting inital $L_i$, we assume that the bot is at the `top-center`. With this assumption, we only need to consider `B & D` metrics for calculating $L_i$



In [22]:
def get_Li(B, D):
    """
    The inital length of strings based on B and D
    """
    return 0.5*np.sqrt(B**2 + D**2)

### TODO
 - [ ] We need to account for 360 degree rotation here
    - Are you talking about the decrease in radius each time the pulley rotates completely?
 - [x] We need to have a reverse calculation done, i.e. given a $x_i,y_i,z_i$ & $x_f,y_f,z_f$ we need to find out what the inital and final thetas. But more on that later.

#### `get_final_string_length` ####
Considering that we have an inital length available, `delta_thetas` given, this function would provide what would be the final length if the pulleys are moved by `delta_theta`

$$ L_f = L_i + r_i\Delta\theta_i $$

#### `get_theta_from_string_length` ####
If we have final length $L_f$ and inital string length $L_i$ and the radius of the pulleys, then we can find the `delta_thetas` moved by the pulleys

<h4 style="color:red">NOTE</h4>
Remember, $L_i$ and & $L_f$ are array/list, so the output would also be array/list each corresponding to individual pulleys.

In [16]:
def get_final_string_length(Li, r, delta_thetas=np.zeros((1,num_pulleys))):
    """
    The final length of strings which need to be achieved for 
    reaching a specific location (based on thetas).
    """
    return Li + r*delta_thetas

def get_theta_from_string_length(Lf, Li, r):
    """
    Given some string lengths Lf and initial lengths Li, return
    the rotation of the pulleys
    """
    return (Lf - Li)/r

#### `get_coordinate` ####
Assumption:
We start from the top of the board, therefore the $L_i$ is same for all strings

1. We take the initial $L_i$ for all strings
2. We are given the `thetas` that the pulley moved
3. We use initial $L_i$ and `thetas` to get final lengths

Then we use following formulas for getting coordinates of the final place the bot moved to

\begin{align}
y &= \frac{l_1^2 - l_2^2 + D^2}{2D} \\
x &= \frac{l_2^2 - l_3^2 + B^2}{2B}
\end{align}

\begin{aligned}
l_1^2 &= (0 - x)^2 + (0 - y)^2 + (H - z)^2 \\ &= x^2 + y^2 + (H - z)^2
\end{aligned}

\begin{aligned}
(H - z)^2 = l_1^2 - x^2 - y^2\\
\end{aligned}

$(H - z)^2$ = Hz2 (Python variable)


In [17]:
def get_coordinate(thetas, B, D, H, r):
    """
    thetas: numpy array of rotations, shape: (num_arrays, )
    B, D, H, r: scalar constants
    """
    Li = get_Li(B, D)
    l = get_final_string_length(Li, r, thetas)
    x = (l[1]**2 - l[2]**2 + B**2)/(2*B)
    y = (l[0]**2 - l[1]**2 + D**2)/(2*D)
    Hz2 = l[0]**2 - x**2 - y**2 # (H-z)^2, eqn 1
    # Is it taut?
    # print(l[0]**2 - l[3]**2, l[1]**2 - l[2]**2)
    z = H - np.sqrt(Hz2)
    return np.array([x, y, z])

### `get_thetas` ###
<p style="color:red">We assume that we always start from the `top` center of the construct that you see on the top.</p>

With the above assumption, the inital length of strings would be taken from `get_Li(B,D)` function.
If we are given the final co-ordinates (x,y,z), we can fund the $Lf$ using following formulas

\begin{align}
\begin{aligned}
l_1^2 &= x^2 + y^2 + (H - z)^2
\end{aligned} \\
\begin{aligned}
l_2^2 &= x^2 + (D - y)^2 + (H - z)^2
\end{aligned} \\
\begin{aligned}
l_3^2 &= (B - x)^2 + (D - y)^2 + (H - z)^2
\end{aligned} \\
\begin{aligned}
l_4^2 &= (B - x)^2 + y^2 + (H - z)^2
\end{aligned} \\
\end{align}

python variable $Hz2 = (H - Z)^2$

In [28]:
def get_thetas(coords, B, D, H, r, back_home=False):
    """
    coords: (x, y, z) coordinates
    B, D, H, r: scalar constants
    """
    l = np.zeros((4,))
    x, y, z = coords
    Hz2 = (H - z)**2 # common term
    l[0] = np.sqrt(x**2 + y**2 + Hz2)
    l[1] = np.sqrt(x**2 + (D - y)**2 + Hz2)
    l[2] = np.sqrt((B - x)**2 + (D - y)**2 + Hz2)
    l[3] = np.sqrt((B - x)**2 + y**2 + Hz2)
    if back_home:
        return get_theta_from_string_length(get_Li(B, D),l, r)
    else:
        return get_theta_from_string_length(l, get_Li(B, D), r)

In [19]:
# Box dimensions
B, D, H = 100, 200, 300
# Pulley radius
r = 0.1
# Pulley locations
Pulleys = [(0, 0, H), (0, D, H), (B, D, H), (B, 0, H)]

Taking some random values of $\Theta$, we can get the corresponding coordinates from `get_coordinates()`.
If $\Theta$ is such that the ropes are taught, we will get back the original $\Theta$ from `get_thetas()`.
If they aren't, the values will be different. Both are demonstrated in the following example. In the first case, $\theta_4$ is clearly different from the value we set, but in the second case, all angles are exactly the same.

### Below is just a cross-check function. ###
1. Each pulley rotated `[10,20,30,40]` rotations respectively, (i.e 1st pulley, 2nd pulley etc.)
2. With these rotations we get the final co-ordinates, remember we assume we start from top of the structure area.
3. While using the final_cordinates we use `get_thetas` function, i.e how much theta is supposed to be moved, this should come back to thetas we decided earlier.

In [20]:
def display(rotations, coords, theta):
    print(f'For rotation = {rotations/(2*np.pi)}')
    x, y, z = coords
    print(f'\tx = {x}, y = {y}, z = {z}')
    print('\tΘ = ', theta)
    print()
    
# Pulleys rotated 10 revolutions, 20, 30, and 40, respectively
rotations = np.array([10, 20, 30, 40])*2*np.pi
coords = get_coordinate(rotations, B, D, H, r)
theta = get_thetas(coords, B, D, H, r)/(2*np.pi)
display(rotations, coords, theta)

# Pulleys rotated 10 revolutions, 20, 20, and 10, respectively
rotations = np.array([10, 20, 20, 10])*2*np.pi
coords = get_coordinate(rotations, B, D, H, r)
theta = get_thetas(coords, B, D, H, r)/(2*np.pi)
display(rotations, coords, theta)

For rotation = [10. 20. 30. 40.]
	x = 41.98822482885034, y = 96.19150450244695, z = 245.88322994588324
	Θ =  [10.        20.        30.        20.5045589]

For rotation = [10. 20. 20. 10.]
	x = 50.0, y = 96.19150450244695, z = 253.18508971098808
	Θ =  [10. 20. 20. 10.]



As we know that all points are calculated from Top, therefore, we need to 2 sets to data.
1. Move from base $(B/2,D/2,H)$ to the target location
2. Move from some location back to $(B/2,D/2,H)$

In this case, we are looking at a dataset which looks like this

|$X_i$|$Y_i$|$Z_i$|$X_f$|$Y_f$|$Z_f$| B | D | H | r |theta_0|theta_1|theta_2|theta_3|
|-----|-----|-----|-----|-----|-----|---|---|---|---|-------|-------|-------|-------|

In [57]:
B = np.arange(1,11,1)
D = np.arange(1,11,1)
H = np.arange(1,11,1)
r = np.arange(0.01,0.11,0.02)

data = np.zeros((1,11))
first=True
element = 0
for b,d,h,r in itertools.product(B,D,H,r):
    x_range = np.arange(0,b,1)
    y_range = np.arange(0,d,1)
    z_range = np.arange(0,h,1)
    for x,y,z in itertools.product(x_range,y_range,z_range):
        thetas_from_home = get_thetas([x,y,z],B=b,D=d,H=h,r=r,back_home=False)
        if first:
            data = np.array([x,y,z,b,d,h,r,thetas_from_home[0],thetas_from_home[1],thetas_from_home[2],thetas_from_home[3]])
            first=False
        else:
            data = np.vstack((data,np.array([x,y,z,b,d,h,r,thetas_from_home[0],thetas_from_home[1],thetas_from_home[2],thetas_from_home[3]])))
            first = False
    if (element+1) % 100 == 0:
        print("Shape : ",data.shape)
    if (element+1) % 1000 == 0:
        print("Completed with %d" %(element))
    element += 1

Shape :  (825, 11)
Shape :  (2750, 11)
Shape :  (5775, 11)
Shape :  (9900, 11)
Shape :  (15125, 11)
Shape :  (16775, 11)
Shape :  (20625, 11)
Shape :  (26675, 11)
Shape :  (34925, 11)
Shape :  (45375, 11)
Completed with 999
Shape :  (47850, 11)
Shape :  (53625, 11)
Shape :  (62700, 11)
Shape :  (75075, 11)


KeyboardInterrupt: 

In [58]:
import pandas as pd
data_df = pd.DataFrame(data,columns=["x_f","y_f","z_f","b","d","h","r","theta0","theta1","theta2","theta3"])

In [59]:
data_df.head(100)

,x_f,y_f,z_f,b,d,h,r,theta0,theta1,theta2,theta3
0,0.0,0.0,0.0,1.0,1.0,1.0,0.01,29.289322,70.710678,102.494403,70.710678
1,0.0,0.0,0.0,1.0,1.0,1.0,0.03,9.763107,23.570226,34.164801,23.570226
2,0.0,0.0,0.0,1.0,1.0,1.0,0.05,5.857864,14.142136,20.498881,14.142136
3,0.0,0.0,0.0,1.0,1.0,1.0,0.07,4.184189,10.101525,14.642058,10.101525
4,0.0,0.0,0.0,1.0,1.0,1.0,0.09,3.254369,7.856742,11.388267,7.856742
5,0.0,0.0,0.0,1.0,1.0,2.0,0.01,129.289322,152.896120,174.238296,152.896120
6,0.0,0.0,1.0,1.0,1.0,2.0,0.01,29.289322,70.710678,102.494403,70.710678
7,0.0,0.0,0.0,1.0,1.0,2.0,0.03,43.096441,50.965373,58.079432,50.965373
8,0.0,0.0,1.0,1.0,1.0,2.0,0.03,9.763107,23.570226,34.164801,23.570226
9,0.0,0.0,0.0,1.0,1.0,2.0,0.05,25.857864,30.579224,34.847659,30.579224
